In [9]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


  Using cached torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_geometric-2.7.0-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.7.0-py3-none-any.whl (1.3 MB)
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl size=1222874 sha256=fc62684ed622b4f9ce58ab309753b02e0b494bae4d40d841b0f839dfacdf9f41
  Stored in directory: /root/.cache/pip/wheels/71/fa/21/bd1d78ce1629aec4ecc924a63b82f6949dda484b6321eac6f2
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp312-cp312-linux_x86_64.whl size=802214 sha256=a994ee18cf1c16b8ee0ef9732a83c2463bd412f94e0968bd6ed19c4e05b48d09
  Stored in directory: /root/.cache/pip/wheels/2e/8f/d0/13408a848

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from sklearn.metrics import f1_score


In [11]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


Processing...
Done!


In [12]:
def train(model, data, epochs=200):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    f1 = f1_score(
        data.y[data.test_mask].cpu(),
        pred[data.test_mask].cpu(),
        average='macro'
    )
    return acc, f1


In [13]:
class MLP(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers):
        super().__init__()
        self.layers = torch.nn.ModuleList()

        self.layers.append(torch.nn.Linear(in_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.layers.append(torch.nn.Linear(hidden_dim, hidden_dim))
        self.layers.append(torch.nn.Linear(hidden_dim, out_dim))

    def forward(self, x, edge_index=None):
        for layer in self.layers[:-1]:
            x = layer(x)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        return self.layers[-1](x)


In [14]:
class GCN(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.convs.append(GCNConv(in_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, out_dim))

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        return self.convs[-1](x, edge_index)


In [15]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.convs.append(SAGEConv(in_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        self.convs.append(SAGEConv(hidden_dim, out_dim))

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        return self.convs[-1](x, edge_index)


In [16]:
class GAT(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers):
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.convs.append(GATConv(in_dim, hidden_dim, heads=8, concat=True))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8, concat=True))
        self.convs.append(GATConv(hidden_dim * 8, out_dim, heads=1, concat=False))

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=0.6, training=self.training)
        return self.convs[-1](x, edge_index)


In [17]:
configs = [
    {"hidden_dim": 16, "layers": 2},
    {"hidden_dim": 32, "layers": 2},
    {"hidden_dim": 64, "layers": 2},
    {"hidden_dim": 64, "layers": 3},
]

models = {
    "MLP": MLP,
    "GCN": GCN,
    "GraphSAGE": GraphSAGE,
    "GAT": GAT
}

results = []

for model_name, model_class in models.items():
    for cfg in configs:
        print(f"Training {model_name} | hidden={cfg['hidden_dim']} layers={cfg['layers']}")
        model = model_class(
            data.num_features,
            cfg["hidden_dim"],
            dataset.num_classes,
            cfg["layers"]
        )
        acc, f1 = train(model, data)
        results.append((model_name, cfg["hidden_dim"], cfg["layers"], acc, f1))


Training MLP | hidden=16 layers=2
Training MLP | hidden=32 layers=2
Training MLP | hidden=64 layers=2
Training MLP | hidden=64 layers=3
Training GCN | hidden=16 layers=2
Training GCN | hidden=32 layers=2
Training GCN | hidden=64 layers=2
Training GCN | hidden=64 layers=3
Training GraphSAGE | hidden=16 layers=2
Training GraphSAGE | hidden=32 layers=2
Training GraphSAGE | hidden=64 layers=2
Training GraphSAGE | hidden=64 layers=3
Training GAT | hidden=16 layers=2
Training GAT | hidden=32 layers=2
Training GAT | hidden=64 layers=2
Training GAT | hidden=64 layers=3


In [19]:
model_formulations = {
    "MLP": {"Enc": "MLP layers (feature-only)", "Gt": "None", "Dec": "Linear layer + softmax", "L": "CrossEntropyLoss"},
    "GCN": {"Enc": "GCNConv layers", "Gt": "Adjacency matrix", "Dec": "Linear layer + softmax", "L": "CrossEntropyLoss"},
    "GraphSAGE": {"Enc": "SAGEConv layers", "Gt": "Edge list", "Dec": "Linear layer + softmax", "L": "CrossEntropyLoss"},
    "GAT": {"Enc": "GATConv layers", "Gt": "Edge list + attention", "Dec": "Linear layer + softmax", "L": "CrossEntropyLoss"}
}

for model_name, model_class in models.items():
    for cfg in configs:
        print(f"Training {model_name} | hidden={cfg['hidden_dim']} layers={cfg['layers']}")
        model = model_class(data.num_features, cfg["hidden_dim"], dataset.num_classes, cfg["layers"])
        acc, f1 = train(model, data)
        print(f"Accuracy: {acc:.3f} | F1: {f1:.3f}")
        formulation = model_formulations[model_name]
        print(f"Enc: {formulation['Enc']} | Gt: {formulation['Gt']} | Dec: {formulation['Dec']} | L: {formulation['L']}\n")


Training MLP | hidden=16 layers=2
Accuracy: 0.476 | F1: 0.481
Enc: MLP layers (feature-only) | Gt: None | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training MLP | hidden=32 layers=2
Accuracy: 0.542 | F1: 0.530
Enc: MLP layers (feature-only) | Gt: None | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training MLP | hidden=64 layers=2
Accuracy: 0.572 | F1: 0.552
Enc: MLP layers (feature-only) | Gt: None | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training MLP | hidden=64 layers=3
Accuracy: 0.579 | F1: 0.564
Enc: MLP layers (feature-only) | Gt: None | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training GCN | hidden=16 layers=2
Accuracy: 0.795 | F1: 0.787
Enc: GCNConv layers | Gt: Adjacency matrix | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training GCN | hidden=32 layers=2
Accuracy: 0.799 | F1: 0.793
Enc: GCNConv layers | Gt: Adjacency matrix | Dec: Linear layer + softmax | L: CrossEntropyLoss

Training GCN | hidden=64 layers=2
Accuracy: 0.804 | F1: 

In [21]:
print("\nFinal Results")
print("Model | Hidden | Layers | Accuracy | F1")
for r in results:
    print(f"{r[0]} | {r[1]} | {r[2]} | {r[3]:.3f} | {r[4]:.3f}")



Final Results
Model | Hidden | Layers | Accuracy | F1
MLP | 16 | 2 | 0.514 | 0.506
MLP | 32 | 2 | 0.552 | 0.544
MLP | 64 | 2 | 0.572 | 0.563
MLP | 64 | 3 | 0.558 | 0.544
GCN | 16 | 2 | 0.815 | 0.808
GCN | 32 | 2 | 0.799 | 0.793
GCN | 64 | 2 | 0.803 | 0.799
GCN | 64 | 3 | 0.801 | 0.797
GraphSAGE | 16 | 2 | 0.792 | 0.788
GraphSAGE | 32 | 2 | 0.791 | 0.779
GraphSAGE | 64 | 2 | 0.805 | 0.799
GraphSAGE | 64 | 3 | 0.749 | 0.736
GAT | 16 | 2 | 0.803 | 0.798
GAT | 32 | 2 | 0.780 | 0.770
GAT | 64 | 2 | 0.653 | 0.661
GAT | 64 | 3 | 0.711 | 0.699
